In [ ]:
!pip install tgt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for tgt: filename=tgt-1.4.4-py3-none-any.whl size=28922 sha256=66a16d6a314968fa7932420be6c61425844bed5417dcdd3d4f65069d1ce41cdc
  Stored in directory: /root/.cache/pip/wheels/e0/d4/e5/039c9c6c19848fcc33223ee8da975d73b2da572d25da27bea7
Successfully built tgt


https://textgridtools.readthedocs.io/en/stable/api.html

Прочитаем TextGrid:

In [ ]:
import tgt

grid = tgt.io.read_textgrid("cta0001.TextGrid")
grid.tiers

[IntervalTier(start_time=0.0, end_time=1.239501133786848, name="sounds", objects=[Interval(0.0, 0.192562358276644, "j"), Interval(0.192562358276644, 0.28816326530612246, "u0"), Interval(0.28816326530612246, 0.3108390022675737, "r'"), Interval(0.3108390022675737, 0.4122448979591837, "i4"), Interval(0.4122448979591837, 0.5647165532879819, "t"), Interval(0.5647165532879819, 0.5882086167800453, "r'"), Interval(0.5882086167800453, 0.7152834467120182, "i0"), Interval(0.7152834467120182, 0.8346031746031746, "f"), Interval(0.8346031746031746, 0.8753741496598639, "a4"), Interval(0.8753741496598639, 0.943718820861678, "n"), Interval(0.943718820861678, 1.0092517006802721, "a4"), Interval(1.0092517006802721, 1.239501133786848, "f")]),
 IntervalTier(start_time=0.0, end_time=1.239501133786848, name="durs zscore", objects=[Interval(0.0, 0.192562358276644, "4.483"), Interval(0.192562358276644, 0.28816326530612246, "0.19"), Interval(0.28816326530612246, 0.3108390022675737, "-1.057"), Interval(0.3108390

Получим названия всех уровней:

In [ ]:
grid.get_tier_names()

['new tier', 'new point tier']

Получим уровень по названию:

In [ ]:
grid.get_tier_by_name("sounds")

IntervalTier(start_time=0.0, end_time=1.239501133786848, name="sounds", objects=[Interval(0.0, 0.192562358276644, "j"), Interval(0.192562358276644, 0.28816326530612246, "u0"), Interval(0.28816326530612246, 0.3108390022675737, "r'"), Interval(0.3108390022675737, 0.4122448979591837, "i4"), Interval(0.4122448979591837, 0.5647165532879819, "t"), Interval(0.5647165532879819, 0.5882086167800453, "r'"), Interval(0.5882086167800453, 0.7152834467120182, "i0"), Interval(0.7152834467120182, 0.8346031746031746, "f"), Interval(0.8346031746031746, 0.8753741496598639, "a4"), Interval(0.8753741496598639, 0.943718820861678, "n"), Interval(0.943718820861678, 1.0092517006802721, "a4"), Interval(1.0092517006802721, 1.239501133786848, "f")])

Создадим пустой TextGrid:

In [ ]:
grid = tgt.core.TextGrid()

Добавим новый IntervalTier:

In [ ]:
new_tier = tgt.core.IntervalTier(name="new tier")
grid.add_tier(new_tier)
new_tier.add_interval(tgt.core.Interval(0, 1.0, "some text"))
new_tier

Добавим новый PointTier:

In [ ]:
new_point_tier = tgt.core.PointTier(name="new point tier")
grid.add_tier(new_point_tier)
new_point_tier.add_point(tgt.core.Point(0.5, "some text"))
new_point_tier

Запишем в разных форматах:

In [ ]:
tgt.io.write_to_file(grid, "new_grid_short.TextGrid", format="short")
tgt.io.write_to_file(grid, "new_grid_long.TextGrid", format="long")

# Задание: написать программу для конвертации TextGrid в .seg и наоборот. (Можно из wav читать частоту дискретизации в идеале, либо задавать в коде)

In [ ]:
def detect_encoding(file_path):
    encoding = "utf-8"
    try:
        l = open(file_path, 'r', encoding="utf-8").read()
        if l.startswith("\ufeff"):
            encoding = "utf-8-sig"
    except UnicodeDecodeError:
        try:
            open(file_path, 'r', encoding="utf-16").read()
            encoding = "utf-16"
        except UnicodeError:
            encoding = "cp1251"
    return encoding

In [ ]:
def read_seg(path):
  with open(path, "r", encoding=detect_encoding(path)) as f:
    lines = [i.strip() for i in f.readlines()]
  return lines

In [ ]:
def write_seg(data, path, s_f=22050, b_p_s=2, code=0, n_c=1, encoding='utf-8'):
  for i in range(len(data[7:])):
    k = i + 7
    data[k] = str(int(data[k].split(',')[0]) / int(data[2].split('=')[1]) / int(data[4].split('=')[1]))+"," + ",".join(data[k].split(',')[1:])
  data[1] = f'SAMPLING_FREQ={s_f}'
  data[2] = f'BYTE_PER_SAMPLE={b_p_s}'
  data[3] = f'CODE={code}'
  data[4] = f'N_CHANNEL={n_c}'
  data[5] = f'N_LABEL={len(data[7:])}'

  for i in range(len(data[7:])):
    k = i + 7
    data[k] = str(int(float(data[k].split(',')[0]) * int(data[2].split('=')[1]) * int(data[4].split('=')[1])))+"," + ",".join(data[k].split(',')[1:])
  with open(path, 'w', encoding=encoding) as fl:
    for i in data:
      fl.write(i+"\n")

In [ ]:
import tgt

grid = tgt.io.read_textgrid("cta0001.TextGrid")
grid.tiers

[IntervalTier(start_time=0.0, end_time=1.239501133786848, name="sounds", objects=[Interval(0.0, 0.192562358276644, "j"), Interval(0.192562358276644, 0.28816326530612246, "u0"), Interval(0.28816326530612246, 0.3108390022675737, "r'"), Interval(0.3108390022675737, 0.4122448979591837, "i4"), Interval(0.4122448979591837, 0.5647165532879819, "t"), Interval(0.5647165532879819, 0.5882086167800453, "r'"), Interval(0.5882086167800453, 0.7152834467120182, "i0"), Interval(0.7152834467120182, 0.8346031746031746, "f"), Interval(0.8346031746031746, 0.8753741496598639, "a4"), Interval(0.8753741496598639, 0.943718820861678, "n"), Interval(0.943718820861678, 1.0092517006802721, "a4"), Interval(1.0092517006802721, 1.239501133786848, "f")]),
 IntervalTier(start_time=0.0, end_time=1.239501133786848, name="durs zscore", objects=[Interval(0.0, 0.192562358276644, "4.483"), Interval(0.192562358276644, 0.28816326530612246, "0.19"), Interval(0.28816326530612246, 0.3108390022675737, "-1.057"), Interval(0.3108390

In [ ]:
from operator import itemgetter
import scipy.io.wavfile as wav

In [ ]:
import scipy.io.wavfile as wav
samplerate, signal = wav.read(file_name)
print(signal.shape)

NameError: ignored

In [ ]:
from itertools import product
letters = "GBRY"
nums = "1234"
levels = [ch + num for num, ch in product(nums, letters)]
print(levels)
level_codes = [2 ** i for i in range(len(levels))]
print(level_codes)
ENUM_LEVEL_CODES = {}
for i in range(len(levels)):
  ENUM_LEVEL_CODES[level_codes[i]] = levels[i] 

['G1', 'B1', 'R1', 'Y1', 'G2', 'B2', 'R2', 'Y2', 'G3', 'B3', 'R3', 'Y3', 'G4', 'B4', 'R4', 'Y4']
[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]


In [ ]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

In [ ]:
def convert_TextGrid_to_Seg(path, wav_path="", byte_per_sample=2, n_channels=1):
  grid = tgt.io.read_textgrid(path)
  print(grid)
  seg_data = ['[PARAMETERS]', 'SAMPLING_FREQ=', f'BYTE_PER_SAMPLE={byte_per_sample}', 
              'CODE=', f'N_CHANNEL={n_channels}', 'N_LABEL=', '[LABELS]',]
  samplerate = 22050
  if wav_path != "":
    samplerate, signal = wav.read(wav_path)
  tier_counter = 1
  tier_letter = "G"
  tier_number = 0
  for tier in grid.tiers:
    tier_counter += 1
    if tier_counter == 5:
      tier_letter = "B"
    if tier_counter == 9:
      tier_letter = "R"
    if tier_counter == 13:
      tier_letter = "Y"
    tier_number = tier_counter % 4
    if tier_number == 0:
      tier_number = 4
    tier_label = tier_letter + str(tier_number)
    tier_label_code = get_key(ENUM_LEVEL_CODES, tier_label)
    if isinstance(tier, tgt.core.IntervalTier):
      print(type(tier))
      k = 0
      for interval in tier:
        start_time = int(round(interval.start_time * samplerate * byte_per_sample * n_channels))
        text = interval.text
        seg_data.append([start_time, tier_label_code, text])
        if k + 1 != len(tier):
          end_time = int(round(interval.end_time * samplerate * byte_per_sample))
          start_time_of_next = int(round(tier[k + 1].start_time * samplerate * byte_per_sample * n_channels))
          if start_time_of_next != end_time:
            seg_data.append([end_time, tier_label_code, ''])
        else:
          end_time = int(round(interval.end_time * samplerate * byte_per_sample))
          seg_data.append([end_time, tier_label_code, "<END>"])
        k += 1
    else:
      print(type(tier))
      for point in tier:
        time = int(round(point.time * samplerate * 2))
        text = point.text
        seg_data.append([time, tier_label_code, text])
  seg_data[7:] = sorted(seg_data[7:], key=itemgetter(0)) # Сортировка меток по времени
  k = 7
  for labels in seg_data[7:]:
    seg_data[k] = ",".join([str(label) for label in labels])
    k += 1
  [print(i) for i in seg_data]
  write_seg(seg_data, f"{path}.seg", s_f=samplerate, b_p_s=byte_per_sample, n_c=n_channels)

In [ ]:
path = "cta0001.TextGrid"
convert_TextGrid_to_Seg(path)

<class 'tgt.core.IntervalTier'>
<class 'tgt.core.IntervalTier'>
[PARAMETERS]
SAMPLING_FREQ=
BYTE_PER_SAMPLE=2
CODE=
N_CHANNEL=1
N_LABEL=
[LABELS]
0,16,j
0,256,4.483
8492,16,u0
8492,256,0.19
12708,16,r'
12708,256,-1.057
13708,16,i4
13708,256,1.64
18180,16,t
18180,256,3.243
24904,16,r'
24904,256,-0.996
25940,16,i0
25940,256,1.528
31544,16,f
31544,256,1.272
36806,16,a4
36806,256,-1.024
38604,16,n
38604,256,0.295
41618,16,a4
41618,256,-0.41
44508,16,f
44508,256,4.764
54662,16,<END>
54662,256,<END>


In [ ]:
read_seg("/content/cta0001.TextGrid.seg")

['[PARAMETERS]',
 'SAMPLING_FREQ=22050',
 'BYTE_PER_SAMPLE=2',
 'CODE=0',
 'N_CHANNEL=1',
 'N_LABEL=26',
 '[LABELS]',
 '0,16,j',
 '0,256,4.483',
 '8492,16,u0',
 '8492,256,0.19',
 "12708,16,r'",
 '12708,256,-1.057',
 '13708,16,i4',
 '13708,256,1.64',
 '18180,16,t',
 '18180,256,3.243',
 "24904,16,r'",
 '24904,256,-0.996',
 '25940,16,i0',
 '25940,256,1.528',
 '31544,16,f',
 '31544,256,1.272',
 '36806,16,a4',
 '36806,256,-1.024',
 '38604,16,n',
 '38604,256,0.295',
 '41618,16,a4',
 '41618,256,-0.41',
 '44508,16,f',
 '44508,256,4.764',
 '54662,16,<END>',
 '54662,256,<END>']

In [ ]:
def convert_Seg_to_TextGrid(path):
  seg = read_seg(path)
  labels = [i.split(",") for i in seg[7:]]
  tiers = set([i[1] for i in labels])
  samplerate = int(seg[1].split('=')[1])
  byte_per_sample = int(seg[2].split('=')[1])
  n_channels = int(seg[4].split('=')[1])
  grid = tgt.core.TextGrid()
  for tier in tiers:
    print(tier)
    tier_name = ENUM_LEVEL_CODES[int(tier)]
    
    new_tier = tgt.core.IntervalTier(name=tier_name)
    grid.add_tier(new_tier)
    labels_of_current_tier = []
    for label in labels:
      if label[1] == tier:
        labels_of_current_tier.append(label)
    for i in range(len(labels_of_current_tier)):
      if i != len(labels_of_current_tier) - 1:
        start_time = int(labels_of_current_tier[i][0]) / samplerate / byte_per_sample / n_channels
        end_time = int(labels_of_current_tier[i + 1][0]) / samplerate / byte_per_sample / n_channels
        new_tier.add_interval(tgt.core.Interval(start_time, end_time, labels_of_current_tier[i][2]))
      else:
        start_time = int(labels_of_current_tier[i][0]) / samplerate / byte_per_sample / n_channels
        new_tier.add_interval(tgt.core.Interval(start_time, start_time, labels_of_current_tier[i][2]))
  
  tgt.io.write_to_file(grid, f"{path}.TextGrid", format="long")

In [ ]:
convert_Seg_to_TextGrid("/content/cta0001.TextGrid.seg")

16
256


In [ ]:
tgt.io.read_textgrid('/content/cta0001.TextGrid.seg.TextGrid').tiers

[IntervalTier(start_time=0.0, end_time=1.239501133786848, name="G2", objects=[Interval(0.0, 0.192562358276644, "j"), Interval(0.192562358276644, 0.28816326530612246, "u0"), Interval(0.28816326530612246, 0.3108390022675737, "r'"), Interval(0.3108390022675737, 0.4122448979591837, "i4"), Interval(0.4122448979591837, 0.5647165532879819, "t"), Interval(0.5647165532879819, 0.5882086167800453, "r'"), Interval(0.5882086167800453, 0.7152834467120182, "i0"), Interval(0.7152834467120182, 0.8346031746031746, "f"), Interval(0.8346031746031746, 0.8753741496598639, "a4"), Interval(0.8753741496598639, 0.943718820861678, "n"), Interval(0.943718820861678, 1.0092517006802721, "a4"), Interval(1.0092517006802721, 1.239501133786848, "f"), Interval(1.239501133786848, 1.239501133786848, "<END>")]),
 IntervalTier(start_time=0.0, end_time=1.239501133786848, name="G3", objects=[Interval(0.0, 0.192562358276644, "4.483"), Interval(0.192562358276644, 0.28816326530612246, "0.19"), Interval(0.28816326530612246, 0.310